In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random as rnd
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
data  = pd.read_csv('ionosphereData.csv', header=None)

headers = ["Attribute"+str(i) for i in range(1,35)]
headers.append("Class") # Add the class label column
data.columns = headers

In [3]:
data.head()

,Attribute1,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,...,Attribute26,Attribute27,Attribute28,Attribute29,Attribute30,Attribute31,Attribute32,Attribute33,Attribute34,Class
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,g
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,b
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,g
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,b
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,g


In [4]:
data.shape

(351, 35)

In [5]:
if (data["Attribute2"] == 0).all():
    print("All values in column 2 are 0.")
    print("Hence the column is redundant and is dropped.")
    data = data.drop(columns=["Attribute2"], axis=1)

else:
    print("Not all values in column 2 are 0")

All values in column 2 are 0.
Hence the column is redundant and is dropped.


In [6]:
data.shape

(351, 34)

In [7]:
columns = data.columns

correlationMatrix = data.corr()

C:\Users\jayad\AppData\Local\Temp\ipykernel_22604\3124951050.py:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlationMatrix = data.corr()


In [8]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values     


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)
print("y_train shape: ", y_train.shape)
print("y_test shape: ", y_test.shape)

X_train shape:  (280, 33)
X_test shape:  (71, 33)
y_train shape:  (280,)
y_test shape:  (71,)


In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)


In [11]:
# Convert 'g' to 1 and 'b' to 0
y_train = (y_train == 'g').astype(int)
y_test = (y_test == 'g').astype(int)


In [12]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward(self, input):
        
        pass

    def backward(self, output_gradient, learning_rate):
        
        pass

In [13]:
class Dense(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(output_size, input_size)
        self.bias = np.random.randn(output_size, 1)

    def forward(self, input):
        self.input = input
        return np.matmul(self.weights, self.input) + self.bias

    def backward(self, output_gradient, learning_rate):
        weights_gradient = np.matmul(output_gradient, self.input.T)
        input_gradient = np.matmul(self.weights.T, output_gradient)
        self.weights -= learning_rate * weights_gradient
        #print("Bias ", self.bias)
        #self.bias -= learning_rate * output_gradient
        return input_gradient

In [14]:
class Activation(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward(self, input):
        self.input = input
        return self.activation(self.input)

    def backward(self, output_gradient, learning_rate):
        return np.multiply(output_gradient, self.activation_prime(self.input))

In [15]:
class tanh(Activation):
    def __init__(self):
        def tanh(x):
            return np.tanh(x)

        def tanh_prime(x):
            return 1 - np.tanh(x) ** 2

        super().__init__(tanh, tanh_prime)

class sigmoid(Activation):
    def __init__(self):
        def sigmoid(x):
            return 1 / (1 + np.exp(-x))

        def sigmoid_prime(x):
            s = sigmoid(x)
            return s * (1 - s)

        super().__init__(sigmoid, sigmoid_prime)

class Relu(Activation):
    def __init__(self):

        def relu(x):
            return np.maximum(0,x)
        
        def relu_prime(x):
            return np.where(x<=0, 0, 1)
        super().__init__(relu, relu_prime)

In [16]:
layers = [Dense(33, 33), tanh(), Dense(33,1), sigmoid()]

epochs = 1000
learning_rate = 0.5



for epoch in range(epochs):

    error = 0
    for i in range(len(X_train)):
        instance = X_train[i].reshape(33,1)
        true_label = y_train[i]
        for layer in layers:
            instance = layer.forward(instance)
        output = instance
        # print(output)
        # print(true_label)
        error += np.square(true_label-output)
        output_gradient = -2*(true_label-output)
        for layer in reversed(layers):
            output_gradient = layer.backward(output_gradient, learning_rate)
    #print("Epoch: ", epoch, "Error: ", error)

In [17]:
def predict(X_test):
    predictions = []
    for instance in X_test:
        instance = instance.reshape(33,1)
        for layer in layers:
            instance = layer.forward(instance)
        predictions.append(instance)
    return predictions

In [18]:
predictions = predict(X_test)

In [19]:
for i in range(len(predictions)):
    predictions[i] = predictions[i]

print("Predictions: ", predictions)

Predictions:  [array([[0.99999831]]), array([[1.]]), array([[0.99999983]]), array([[0.99944217]]), array([[0.99999897]]), array([[0.99970545]]), array([[0.99950269]]), array([[1.]]), array([[0.99987925]]), array([[1.]]), array([[9.31416762e-06]]), array([[0.99405728]]), array([[8.03366972e-13]]), array([[0.99986412]]), array([[0.99997025]]), array([[0.98495557]]), array([[0.99999998]]), array([[0.00172757]]), array([[0.99999013]]), array([[3.06171704e-07]]), array([[0.99879631]]), array([[7.55545113e-11]]), array([[2.24422977e-10]]), array([[0.99822859]]), array([[0.99999522]]), array([[0.74185073]]), array([[5.5747599e-07]]), array([[1.05240779e-07]]), array([[9.2154721e-05]]), array([[0.99985267]]), array([[0.99564583]]), array([[0.99999911]]), array([[0.99999998]]), array([[0.99999999]]), array([[1.]]), array([[0.00019206]]), array([[0.99999496]]), array([[0.99999506]]), array([[0.95615689]]), array([[0.99999915]]), array([[1.]]), array([[0.99999931]]), array([[0.99999999]]), array(

In [20]:
preds = []
for i in predictions:
    if i >= 0.5:
        preds.append(1)
    else:
        preds.append(0)


In [21]:
def accuracy(y_test, predictions):
    correct = 0
    for i in range(len(y_test)):
        if y_test[i] == predictions[i]:
            correct += 1
    return correct/len(y_test)

In [22]:
def confusionMatrix(predictions, y_test):

    # TP => Predictions[i]=1 and y_test[i]=1
    # TN => Predictions[i]=0 and y_test[i]=0
    # FP => Predictions[i]=1 and y_test[i]=0
    # FN => Predictions[i]=0 and y_test[i]=1
    TP, TN, FP, FN = 0, 0, 0, 0

    for i in range(len(predictions)):
        if predictions[i]==1 and y_test[i]==1:
            TP += 1
        elif predictions[i]==0 and y_test[i]==0:
            TN += 1
        elif predictions[i]==1 and y_test[i]==0:
            FP += 1
        else:
            FN += 1
    
    return [[TP, FP],[FN,TN]]

In [23]:
def precision(confusion_matrix):
    return confusion_matrix[0][0]/(confusion_matrix[0][0]+confusion_matrix[0][1])


def truepositiveRate(confusion_matrix):
    return confusion_matrix[0][0]/(confusion_matrix[0][0]+confusion_matrix[1][0]) # sensitivity

In [24]:
print(accuracy(y_test, preds))

0.8450704225352113


In [25]:
print(precision(confusionMatrix(preds, y_test)))

0.9767441860465116


In [26]:
print(confusionMatrix(preds, y_test))

[[42, 1], [10, 18]]
